some observations    
===============

**vocabulary size**    
tweet tokenizer / no preprocessing = 313803    
tweet tokenizer / with cleaning method = 260580    
tweet tokenizer / with cleaning method, reduce length = 240963

## Create the Vocab Set

In [3]:
import pickle
import pandas as pd
import math
from collections import Counter
import sys
import csv
import string
import re
import emoji
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from itertools import chain
from indexer import TwitterIQ

In [ ]:
inv_index = TwitterIQ('tweets.csv')

In [2]:
df = pd.read_csv('tweets.csv', sep='\t', usecols=[1,4], names=['id', 'tweet'])

In [3]:
tweets = df['tweet']

In [4]:
tokenizer = TweetTokenizer(reduce_len=True)
unicodes2remove = [
    # all kinds of quotes
    u'\u2018', u'\u2019', u'\u201a', u'\u201b', u'\u201c', \
    u'\u201d', u'\u201e', u'\u201f', u'\u2014',
    # all kinds of hyphens
    u'\u002d', u'\u058a', u'\u05be', u'\u1400', u'\u1806', \
    u'\u2010', u'\u2011', u'\u2012', u'\u2013',
    u'\u2014', u'\u2015', u'\u2e17', u'\u2e1a', u'\u2e3a', \
    u'\u2e3b', u'\u2e40', u'\u301c', u'\u3030',
    u'\u30a0', u'\ufe31', u'\ufe32', u'\ufe58', u'\ufe63', \
    u'\uff0d', u'\u00b4'
]

punctuation = string.punctuation.replace('@', '') + ''.join(unicodes2remove)
# regex to match urls (taken from the web)
urlregex = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]'
                           '|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
# keep @ to be able to recognize usernames
punctuation = string.punctuation.replace('@', '') + ''.join(unicodes2remove)
punctuation = punctuation.replace('#', '')
# a bunch of emoji unicodes
emojis = ''.join(emoji.UNICODE_EMOJI)
emojis = emojis.replace('#', '')
# combined english and german stop words
stop_words = set(stopwords.words('english') + stopwords.words('german'))

In [96]:
def clean(s):
    """
    Normalizes a string (tweet) by removing the urls, punctuation, digits,
    emojis, by putting everything to lowercase and removing the
    stop words. Tokenization is performed aswell.

    :param s the string (tweet) to clean
    :return: returns a list of cleaned tokens
    """
    s = s.replace('[NEWLINE]', '')
    s = s.replace('…', '...')
    s = urlregex.sub('', s).strip()
    s = s.translate(str.maketrans('', '', punctuation + string.digits \
                                  + emojis)).strip()
    s = s.lower()
    s = tokenizer.tokenize(s)
    #s = [w for w in s if w not in stop_words]
    return s

In [97]:
#cleans and tokenizes
tokenized = df['tweet'].apply(clean)

In [98]:
tokenized

0         [@knakatani, @chikonjugular, @joofford, @steve...
1         [@fischerkurt, lady, whats, a, tumor, #kippcha...
2         [@kingsofmetal, ohne, diagnoseverdacht, ist, e...
3         [@germanletsplay, @quentin, @lopoopl, @levanni...
4         [interesting, pcr, rate, at, major, centers, a...
5         [new, nanobots, kill, cancerous, tumors, by, c...
6         [rip, salem, aufgrund, eines, tumors, eingesch...
7         [cancerfighting, nanorobots, programmed, to, s...
8          [@riptear, tumorsdat, leg, is, straight, mccain]
9         [quote, this, with, one, statement, about, can...
10        [@joyannreid, @pampylu, you, are, wrong, if, y...
11        [erstmal, nen, anti, cancer, stick, in, die, l...
12        [#usa, #upi, #news, broadcast, by, #emetnewspr...
13        [leseempfehlung, extraordinary, correlation, b...
14                    [@fazefuzzface, welcome, to, obesity]
15        [@isonnylucas, thats, exactly, the, point, of,...
16                [obese, adult, free, o

In [109]:
V = set(chain.from_iterable(tokenized))

In [110]:
V

{'filmstarauch',
 '#siemensehs',
 'kopeks',
 '#softshell',
 'ah',
 'racial',
 'geschäfts',
 '@thenation',
 '#markusmajowski',
 'nettdu',
 'youtubebajjuffe',
 'progressivepoprock',
 'tttt',
 '@shehabifares',
 'itconsultant',
 'kommata',
 '#fierce',
 'shiftcmd',
 'krapfenzeit',
 '#magisch',
 'kojima',
 'فقط',
 '@ruthciesinger',
 'längeres',
 '@andiwheeler',
 '@mtin',
 'eliminate',
 'propagandadelikte',
 '@simplyimagine',
 'sommerloch',
 'turbocharged',
 '@bilalkaya',
 'knubbel',
 '@margaowski',
 'kraftdas',
 '#angelengine',
 'klettern',
 '#buchde',
 '#osternfacebookthe',
 '#eigernordwand',
 'ermutigen',
 '#himmelfahrt',
 '@boedefeld',
 'umhüllt',
 'adoring',
 'gesandten',
 'endgeräte',
 '#falscherjahrgang',
 '#betheyouyouwanttosee',
 '#fusion',
 '#platelets',
 'gratfull',
 'nachrüstung',
 '#wildpferd',
 'odia',
 'hilfesuchen',
 'oncircumstances',
 'abschaltet',
 '@skaadooosh',
 'beklommen',
 'maskotchen',
 '@rycet',
 'kesinkes',
 'diavolo',
 'wageningen',
 'tellement',
 '#spilledink',
 '

In [111]:
len(V)

241328

## Pickle

In [112]:
#dump
with open('vocabulary_naive.pickle', 'wb') as f:
    pickle.dump(V, f)

In [113]:
#load

f = open('vocabulary_naive.pickle', 'rb')
V = pickle.load(f)
f.close()

## TF-IDF

In [1]:
def tfidf(doc1, doc2, V, tweets):
    """
    d1,d2 -> list of tokens
    V: Vocabulary
    tweets: 
    """
    #i think we can use this intersection here because words that are in the in one 
    #vector but not in the other result in zero multiplication
    intersect = set(doc1).intersection(set(doc2))
    if not intersect:
        return (pd.DataFrame({}), pd.DataFrame({}))
        
    c1 = Counter(doc1)#token count
    c2 = Counter(doc2)#token count
    d1 = {}
    d2 = {}
    for t in doc1:
        if t not in d1.keys() and t in intersect:
            #tf idf
            tf = c1[t]
            df = len([tweet for tweet in tweets if t in tweet]) #naive string matching
            if df == 0:
                continue
            else:
                idf = len(tweets) / df
                tfidf = (1 + math.log10(tf)) * (math.log10(idf))
            d1[t] = tfidf
    for t in doc2:
        if t not in d2.keys() and t in intersect:
            #tf idf
            tf = c2[t]
            #naive string matching
            #TODO: after tokenization and preprocessing was improved 
            #we should look if the token is in a tweet by comparing single tokens and not 
            #the whole string
            df = len([tweet for tweet in tweets if t in tweet])
            if df == 0:
                continue
            else:
                idf = len(tweets) / df
                tfidf = (1 + math.log10(tf)) * (math.log10(idf))
            d2[t] = tfidf
            
    df_tfidf = pd.DataFrame().from_dict(d1, orient='index')
    df_tfidf[1] = pd.DataFrame().from_dict(d2, orient='index')
    return df_tfidf

In [115]:
def cosine(vec1, vec2):
    if len(vec1) == 0 or len(vec2) == 0:
        return 0
    nominator = 0
    denominator = 0
    vec1_length = 0
    vec2_length = 0
    for v1,v2 in zip(vec1,vec2):
        nominator += v1*v2
        vec1_length += v1*v1
        vec2_length += v2*v2
    vec1_length = math.sqrt(vec1_length)
    vec2_length = math.sqrt(vec2_length)
    denominator = vec1_length * vec2_length 
    return nominator / denominator

### Examples
Execute the example docs you want to compare.

#### 1

In [31]:
#@Brandon: its fun to play around with the those documents
doc1 = "this is a random tweet Hausarzt Affe Affe Affe".split()
doc2 = "this is random a a a a a tweet Hausarzt Hausarzt Hausarzt Hausarzt Hausarzt I think bla foo Affe".split()

#### 2

In [70]:
doc1 = "i don't think society understands how hurtful it is when this kind of behavior by the POTUS becomes an accepted form of political discourse".split()
doc2 = 'And it is grievously hurtful to our society when vilification becomes an accepted form of political debate and negative campaigning becomes a full-time occupation.'.split()

#### 3

In [42]:
doc1 = "He was a sk8er boi, she said see you later boy".split()
doc2 = "I'm with the sk8er boi, I said see you later boy".split()

#### 4

In [68]:
doc1 = 'and she told me Ich sitze noch in der Küche'.split()
doc2 = 'Was meinst du mit sitting here with nachos'.split()

### Compute scores

In [71]:
df_tfidf = tfidf(doc1, doc2, V, tweets)
df_tfidf

,0,1
society,3.302769,3.302769
hurtful,4.381950,4.381950
it,0.638535,0.638535
is,0.601611,0.601611
when,2.063470,2.063470
of,1.389663,1.068125
becomes,3.850471,5.009579
an,0.512050,0.512050
accepted,3.837882,3.837882
form,2.270352,2.270352


In [72]:
cosine(df_tfidf[0], df_tfidf[1])

0.9933358489295645

In [116]:
def top_x(x, q, tweets, V):
    """
    x: top x number
    q: query to compare to
    tweets: all the tweets -> assumed to be cleaned/tokenized
    """
    q = clean(q)
    print(q)
    for tweet in tweets:
        print(set(tweet) & set(q))
        df_tfidf = tfidf(q, tweet, V, tweets)
        #print(df_tfidf)
        cos = cosine(df_tfidf[0], df_tfidf[1])
        print(cos)

In [120]:
t3 = """
Krankheit
Zur Navigation springenZur Suche springen

Das kranke Mädchen (Den syge pige) von Michael Ancher, 1882
Krankheit ist ein Zustand verminderter Leistungsfähigkeit, der auf Funktionsstörungen von einem oder mehreren Organen, der Psyche oder des gesamten Organismus beruht und zurückgeht. Diese Störungen werden ihrerseits durch strukturelle Veränderungen von Zellen und Geweben hervorgerufen.[1][2]

Die Lehre von den Krankheiten ist die Pathologie, während die Nosologie sich mit der systematischen Einteilung von Krankheiten beschäftigt.[2]


Inhaltsverzeichnis
1	Wortherkunft
2	Definition
2.1	Krankheit und Gesundheit
2.2	Medizin
2.3	Recht
3	Geschichtliche und kulturelle Aspekte
4	Typische Reaktionen bei schwerer Krankheit
5	Systematik
6	Ursachen und Verlauf
7	Krankheitsmodell
8	Literatur
9	Weblinks
10	Einzelnachweise
Wortherkunft
Das Wort Krankheit, von mittelhochdeutsch krancheit bzw. krankeit,[3] bedeutete ursprünglich vor allem ‚Schwäche‘ und ist abgeleitet von mittelhochdeutsch kranc (mit den Bedeutungen ‚schwach, kraftlos, hinfällig, geschwächt‘ usw.). Zugrunde liegen wohl westgermanisch kranka (‚hinfällig‘; vgl. althochdeutsch chrancholōn: ‚schwach werden, straucheln‘, und angelsächsisch cringan, ‚sich winden, im Kampf niederstürzen, hinfällig sein‘) und indogermanisch grengh- (‚ringeln‘).[4][5][6]

Definition
Krankheit und Gesundheit
→ Hauptartikel: Medizintheorie#Krankheit und Gesundheit
Krankheit wird oft im Gegensatz zu Gesundheit definiert. Die Weltgesundheitsorganisation (WHO) hat allerdings Gesundheit auch schon 1946[7] als idealen Zustand optimalen Wohlbefindens definiert. Zudem ist Krankheit nicht die einzige mögliche Ursache für mangelhafte Gesundheit.[8] Die Übergänge zwischen „Gesundheit“ und „Krankheit“ sind fließend. Vieles mag letztlich einfach eine Frage der Sichtweise sein, zumal der Ausdruck Krankheit keine biologische Konstante, sondern ein kulturelles wertbezogenes Konstrukt[9] darstellt. So hat sich der Begriff Befindlichkeitsstörung für Einschränkungen des leiblichen oder seelischen Wohlbefindens ohne objektivierbaren medizinischen Krankheitswert eingebürgert. Andererseits können als krankhaft definierbare Zustände auch ohne subjektiven Leidensdruck vorliegen.

Die normale Funktion eines Organismus ergibt sich aus der Regelhaftigkeit der Lebensvorgänge; in unterschiedlichem Ausmaß beinhaltet sie die Fähigkeit zur Anpassung an veränderte innere und äußere Bedingungen. Ihre Beurteilung durch Menschen weist auch Abhängigkeit von deren Normvorstellungen auf.

Als Funktionsstörung kann Krankheit verschiedene Bereiche lebendigen Seins betreffen und sich in deren Wechselwirkungen entwickeln. Physiologische Funktionen sind wesentliche Eigenschaft des Lebens. Organismen existieren in komplexen Umwelten und erhalten, erneuern und verändern sich durch beständigen stofflichen und energetischen Austausch. Viele Arten von Organismen leben in sozialen Zusammenhängen. Zu den Funktionen des Lebens gehört auch Verhalten und höherentwickelte Organismen weisen emotionale Funktionen auf. Die Personalität und Sozialität von Menschen funktioniert auch in Abhängigkeit von ihrer kulturellen Welt.

Die Zuordnung von Erkrankungen eines konkreten Lebewesens zu abstrakten „Krankheitskategorien“ gilt als wichtig im Zusammenhang mit der Entwicklung von Ansätzen zur Behandlung und ihren verwaltungsmässigen und wirtschaftlichen Rahmenbedingungen. Eine zu diesem Zweck entwickelte Systematik ist die Internationale Klassifikation der Krankheiten und verwandter Gesundheitsprobleme (ICD-10).[10]

Medizin
Seit Jahrhunderten ist die Medizin bestrebt, den allgemeinen Krankheitsbegriff eindeutig zu definieren und abzugrenzen.[11][12] Dabei hat sie sich mit verschiedenartigen Krankheitsbildern und konkreten Erkrankungen auseinanderzusetzen.[13]

Zur Erkennung von Krankheiten bei individuellen Patienten bedarf es entsprechender Untersuchungen (Diagnostik). Damit werden Befunde erhoben, welche der Erstellung einer Diagnose dienen können. Hat ein Mensch das Gefühl, „krank“ zu sein, oder ist bei jemandem eine Krankheit bereits erkannt worden, spricht man in der Medizin von einem Patienten.

Einzelne Beschwerden eines Patienten können Symptome definierbarer Krankheiten sein. Mehrere typischerweise gleichzeitig auftretende Symptome werden als Syndrom (Symptomkomplex) bezeichnet. Symptome oder Symptomenkomplexe, die auf eine gemeinsame Ursache (Ätiologie) zurückführbar sind, lassen die Bestimmung einer spezifischen Krankheit (Morbus) im Sinne der modernen Medizin zu (siehe Pathogenese). Eindeutig scheint dies, wenn notwendige oder hinreichende Krankheitsursachen feststellbar sind. Für definierte Infektionskrankheiten sind z. B. spezifische Krankheitserreger notwendig; manche angeborene Krankheiten treten zwingend bei bestimmten molekulargenetischen Veränderungen auf.

Oftmals sind Krankheiten aber auch nicht eindeutig auf nachweisbare Ursachen zurückzuführen. Mitunter werden sie dann durch regelhaft vorliegende strukturelle bzw. funktionelle Erscheinungen definiert. Die Gesamtheit aller für eine Krankheit typischen Erscheinungen ist das Krankheitsbild (Synonym: Entität), das in mehr oder weniger unterschiedlichen Ausformungen beobachtet werden kann. Die Lehre von den Krankheiten ist die Pathologie.

Recht
Der Umstand, dass die Übergänge zwischen „Gesundheit“ und „Krankheit“ fließend sind, wirft auch juristische Probleme auf. Der Begriff „Krankheit“ selbst wird inhaltlich heftig diskutiert, insbesondere im sozialversicherungsrechtlichen Bereich.[14]

Krankheit im Sinne des Sozialversicherungsrechts ist eine Störung des körperlichen oder seelischen Wohlbefindens, somit eine Abweichung von der Norm „Gesundheit“. (vgl. § 120 Abs. 1 Ziffer 1 ASVG, wonach Krankheit „ein regelwidriger Körper- oder Geisteszustand ist, der die Krankenbehandlung notwendig macht“.)

Der Bundesgerichtshof (BGH) hat am 21. März 1958 definiert: „Krankheit ist jede Störung der normalen Beschaffenheit oder der normalen Tätigkeit des Körpers, die geheilt, d. h. beseitigt oder gelindert werden kann.“[15] Nach einer neueren Formulierung des Bundessozialgerichts (BSG) wird im Kranken- und Unfallversicherungswesen unter Krankheit „ein regelwidriger Körper- oder Geisteszustand, der ärztlicher Behandlung bedarf und/oder Arbeitsunfähigkeit zur Folge hat“ verstanden.[16] Dadurch ist der medizinische Krankheitsbegriff nicht deckungsgleich mit dem sozialrechtlichen. Entscheidende Kriterien für die Beurteilung als Krankheit im Sozialrecht sind:

Behandlungsbedürftigkeit (nicht bei altersbedingten Erscheinungen; kosmetischen Behandlungen, die rein ästhetischer Natur sind (wie beispielsweise Haartransplantation), sehr wohl jedoch, wenn eine anerkannte medizinische Notwendigkeit vorliegt (wie beispielsweise Korrektur der Nasenscheidewand oder Behandlung von Narben))
Wahrnehmbarkeit nach außen (z. B. Disharmonien der genetischen Werte erfüllen den Sachverhalt nicht)
Besserung des Leidens oder Verhütung von Verschlimmerungen (die Behandlung muss nach den Grundsätzen der ärztlichen Wissenschaft erfolgversprechend sein)
Der letzte Punkt kann problematisch für unheilbare Krankheiten sein, da in solchen Fällen keine Besserung möglich ist.

Davon ist im Sozialversicherungsrecht das Gebrechen (§ 154 ASVG) abzugrenzen. Dabei handelt es sich um unbehebbare Leiden, deren Entwicklung abgeschlossen ist und eine Möglichkeit auf ärztliche Einflussnahme im Sinne einer Heilung, Besserung oder Verhütung von Verschlimmerungen nicht möglich ist. Beschwerden durch Unfälle und deren Folgen werden in der Schweiz aus juristischer Sicht nicht dem Begriff „Krankheit“ zugerechnet.[17]

Geschichtliche und kulturelle Aspekte

Porträt des Feldmarschalls Alessandro Marchese del Borro († 1656)[18]
Mit Beginn der Neuzeit[19] wurde Krankheit zunehmend als Störung des Organismus begriffen.[20] Die Einordnung, das Maß der „Normalität“ überschreitender Veränderungen eines Menschen, hängt stark von der Kultur und der Epoche ab.[21][22] So war Fettleibigkeit (Adipositas) in der Renaissance ein Status-Symbol, heutzutage wird sie allgemein als krankhaft betrachtet.[23]

Dass bestimmte chemische Elemente die Grundbestandteile von lebenden Organismen sind, war zu Beginn des 19. Jahrhunderts dann auch Teil eines medizinischen Konzeptes des französischen, an die Entwicklungen in der Chemie seiner Zeit anknüpfenden Arztes Jean Baptiste Thimotée Baumes (1756–1828). Nach dessen 1806 publizierten[24] Ansichten reagieren diese Elemente entsprechend ihrer chemischen Affinität im Körper. Krankheiten seien demnach neben Störungen im Wärmehaushalt und Wasserhaushalt auch solche des Stickstoffhaushalts oder Phosphorhaushalts.[25]

Typische Reaktionen bei schwerer Krankheit
Vor allem schwere Krankheit muss nicht nur geistig, sondern auch emotional verarbeitet werden. Für die Auseinandersetzung mit einer Erkrankung gibt es ganz typische Reaktionsweisen:[26]

Rückzug in die kindliche Abhängigkeit: Diese Regression kann einerseits gut sein und die Energiereserven schonen, andererseits aber auch in übermäßige Forderung von Aufmerksamkeit und Obsorge gipfeln.
Verleugnung: Die Krankheit wird verleugnet und damit auch ein guter Umgang damit verhindert.
Rationalisierung und Verschiebung: Die Probleme der Krankheit werden auf andere Ursachen geschoben und die Krankheit als Ursache verleugnet.
Angstreaktionen und depressive Reaktionen
Systematik
→ Hauptartikel: Nosologie
Systematische Einteilung von Krankheiten wird als Nosologie (Krankheitslehre) bezeichnet. Die Bezeichnungen der Krankheiten, die Abgrenzung einzelner Krankheitsbilder (Entitäten) gegeneinander und die Systematik der Krankheiten sind ständigem Wandel unterworfen[27][28] (vgl. Liste historischer Krankheitsbezeichnungen). Die moderne Einteilung der Krankheiten im medizinischen Krankheitsmodell kann grob organbezogen nach den Hauptdiagnosegruppen (Major Diagnostic Categories, MDC) erfolgen.

Eine genauere Einteilung erlaubt die Internationale Klassifikation der Krankheiten (ICD-10), bzw. für den onkologischen Bereich entsprechend der ICD-O.

Eine an den bekannten oder vermuteten Ursachen orientierte Einteilung ist die nach

Erbkrankheiten (durch Vererbung verursacht)
Infektionskrankheiten (durch Infektionen verursacht)
Kinderkrankheiten
Unfälle und Verletzungen (durch Außeneinwirkung verursacht)
Vergiftungen, Verätzungen, Verbrennungen und andere durch chemische oder physikalische Noxen verursachte Schädigungen
Degenerative Erkrankungen (durch Abnutzung von Organen oder Zellen verursacht)
Autoimmunkrankheiten (das Immunsystem bekämpft gesunde Zellen)
Tumorkrankheiten (Zellen entarten und wachsen unkontrolliert)
Iatrogene Erkrankungen (durch Behandlung verursacht)
Psychische Erkrankungen
Soziale Krankheiten
Zivilisationskrankheiten (falsche Ernährung, Bewegungsmangel u. a.)
Mangelerkrankungen
Krankheiten, die durch Nahrungsmittel verursacht werden.
Eigenschaften, die statistisch für sich alleine betrachtet die Rate des Auftretens bestimmter Krankheitsbilder erhöhen, ohne ihrem Wesen nach für diese alleinig verantwortlich zu sein, werden als sogenannte Risikofaktoren bezeichnet. Als klassisches Beispiel sei hierzu die positive statistische Korrelation zwischen der Erhöhung des Blutdruckes und dem Auftreten kardiovasculärer Erkrankungen angeführt.

Eine seelisch-körperliche Betrachtungs- und Heilweise, unter Berücksichtigung der emotionalen und sozialen Ursachen sowie der Persönlichkeit und des Lebensschicksals des Patienten versucht die psychosomatische Medizin. Neben der evidenzbasierten Medizin gibt es auch andere Betrachtungsweisen zu Krankheitsursachen bis hin zu Theorien über metaphysische und esoterische Zusammenhänge. Mit der gesellschaftlichen Bedingtheit von Erkrankung und Krankheitsverläufen sowie der staatlichen Steuerung des Gesundheitswesens beschäftigt sich die Medizinsoziologie.

Ursachen und Verlauf
→ Hauptartikel: Ätiologie (Medizin) und Krankheitsverlauf
Als Ursachen für Erkrankungen werden in der modernen Medizin nennenswert abweichende Veränderungen vom gesunden Zustand von Teilen des Körpers betrachtet – und damit auch deren Funktion, sogenannte organpathologische Befunde.

Siehe auch: Pathologie, Psychopathologie und Phytopathologie
Die Ursachen für diese Veränderungen lassen sich in innere und äußere Faktoren einteilen. Zu den inneren Faktoren gehören das allgemeine Altern, Erbkrankheiten und ererbte Anfälligkeiten/Anlagen, embryonale Fehlbildungen sowie psychische Erkrankungen. Diese sind wenig beeinflussbar. Demgegenüber sind äußere Faktoren, wie soziale Verhältnisse, Stress, Ernährung, Umweltbedingungen und Krankheitserreger gut beeinflussbar.

Krankheit führt – behandelt oder unbehandelt – zu Heilung, Remission, einem Rezidiv (oder mehreren Rezidiven), Leiden oder Tod.

Häufig verwendete Begriffe, die den zeitlichen Verlauf beschreiben, sind:

akut – plötzlicher Beginn, z. B. grippaler Infekt
subakut – allmählicher Beginn, z. B. Hepatitis B
chronisch – längerer Verlauf, z. B. Multiple Sklerose
chronisch rezidivierend – immer wieder auftretend, z. B. „chronische“ Bronchitis
foudroyant bzw. fulminant – „blitzartig“ einsetzend und schwer verlaufend, z. B. Sepsis
Krankheitsmodell
Unter einem Krankheitsmodell versteht man einen wissenschaftstheoretischen Ansatz, mit dem Ziel, in modellhaft vereinfachter Form eine Krankheit zu erklären.

Eine Diskussion um Krankheitsmodelle ist aus der Frage entstanden, welches der objektive Unterschied zwischen normal und anormal, als krankhaft sei. Diese Unterscheidung betrifft meist nicht das Urteil des Kranken selbst, sondern das seiner Umgebung. Es ist auf den vermeintlich Kranken gerichtet und gibt die Auffassungen der nächsten Angehörigen und des sozialen Umfeldes über Krankheit wieder. Es umfasst somit auch einen soziologischen und epidemiologischen Aspekt, der z. B. in der Medizinsoziologie und in der Sozialpsychiatrie von Interesse ist.

Ein weiterer Ansatz betrifft die Kontroverse zwischen durchgehendem und uneinheitlichem Behandlungsansatz. Der durchgehende Ansatz besagt, dass ein einheitliches gesundheitliches Erklärungsprinzip sowohl für Gesunde als auch Kranke ausreiche. Das uneinheitliche Prinzip besagt, dass für Kranke besondere eigengesetzliche Prozesse ablaufen, die einer spezialisierten Behandlung je nach Art des festgestellten Falles bedürfen. Die Forderung nach einem einheitlichen Behandlungsprinzip geht auf die Forderung von Ludolf von Krehl zurück, dass der Arzt nicht verschiedene Krankheiten behandeln solle, sondern eher den Kranken als Person im Auge zu halten habe. Dieses Prinzip trägt sehr zur Vermenschlichung der Krankenbehandlung bei und nimmt dem Kranken das gesellschaftliche Stigma des Abnormen und Unverständlichen.[29]

Literatur
Emanuel Berghoff: Entwicklungsgeschichte des Krankheitsbegriffes. (= Wiener Beiträge zur Geschichte der Medizin. 1). Wien 1947.
Rudolf Degkwitz, Helmut Siedow (Hrsg.): Zum umstrittenen psychiatrischen Krankheitsbegriff. (= Standorte der Psychiatrie. Band 2). Urban & Schwarzenberg, München 1981, ISBN 3-541-07972-X.
Dietrich von Engelhardt: Krankheit, Krankheitsbegriff (Neuzeit). In: Werner E. Gerabek, Bernhard D. Haage, Gundolf Keil, Wolfgang Wegner (Hrsg.): Enzyklopädie Medizingeschichte. de Gruyter, Berlin 2005, ISBN 3-11-015714-4, S. 797–803.
William Heberden (Junior): Commentaries on the history and cure of diseases. London 1802. (Neudruck mit einer Einführung von Paul Klemperer. New York 1962)
Johannes Kiesel: Was ist krank? Was ist gesund? Zum Diskurs über Prävention und Gesundheitsförderung. Campus Verlag, Frankfurt 2012, ISBN 978-3-593-39786-3.
Ingo-Wolf Kittel: Systematische Überlegungen zum Begriff „krank“ … (1981; 2001 ern. sgipt.org)
Rainer Lutz: Gesundheit und Genuss: Euthyme Grundlagen der Verhaltenstherapie. In: J. Margraf: Lehrbuch der Verhaltenstherapie. Band 1, Springer, Berlin 1996, ISBN 3-540-60378-6.
Karl Eduard Rothschuh: Konzepte der Medizin in Vergangenheit und Gegenwart. Hippokrates, Stuttgart 1978, ISBN 3-7773-0442-5.
Hermann Metzke: Lexikon der historischen Krankheitsbezeichnungen. Degener & Co., Insingen 1995, ISBN 3-7686-1051-9.
Karl Eduard Rothschuh (Hrsg.): Was ist Krankheit? Wissenschaftliche Buchgesellschaft, Darmstadt 1975, ISBN 3-534-06021-0.
Hans Schaefer: Der Krankheitsbegriff. In: Maria Blohmke u. a. (Hrsg.): Handbuch der Sozialmedizin. Band III, Stuttgart 1976, ISBN 3-432-87651-3.
Willi Seitz: Verhaltensstörungen. In: Dieter Rost: Handwörterbuch Pädagogische Psychologie. 2. Auflage. PVU, Weinheim 2001, ISBN 3-621-27491-X.
Susan Sontag: Krankheit als Metapher. Fischer, Frankfurt 1981, ISBN 3-596-23823-4.
Weblinks
 Commons: Krankheiten – Sammlung von Bildern, Videos und Audiodateien
 Wiktionary: Krankheit – Bedeutungserklärungen, Wortherkunft, Synonyme, Übersetzungen
 Wikiquote: Krankheit – Zitate
 Wikinews: Krankheit – in den Nachrichten
 Wikisource: Krankheit – Quellen und Volltexte
Sebastian Brändli: Krankheit. In: Historisches Lexikon der Schweiz.
Einzelnachweise
 Hugo Ribbert: Die Bedeutung der Krankheiten für die Entwicklung der Menschheit. Friedrich Cohen, Bonn 1912, S. 1 f.
 Pschyrembel klinisches Wörterbuch, Verlag deGruyter, 267. Auflage 2017 (ISBN 978-3-11-049497-6). (Stichwort Krankheit, online)
 Wolfgang Pfeifer et al. (2005): Etymologisches Wörterbuch des Deutschen. 8. Auflage. Deutscher Taschenbuch Verlag, ISBN 3-423-32511-9, Seite 727, Eintrag „krank“.
 Friedrich Kluge, Alfred Götze: Etymologisches Wörterbuch der deutschen Sprache. 20. Aufl., hrsg. von Walther Mitzka, De Gruyter, Berlin/ New York 1967; Neudruck („21. unveränderte Auflage“) ebenda 1975, ISBN 3-11-005709-3, S. 400 f.
 Heinrich Schipperges: Die Kranken im Mittelalter. München 1990, S. 70.
 Lutz Mackensen: Deutsche Etymologie. Ein Leitfaden durch die Geschichte des deutschen Wortes. (Bremen 1962) Berlin/Darmstadt/Wien 1966, S. 44.
 „Die Gesundheit ist ein Zustand des vollständigen körperlichen, geistigen und sozialen Wohlergehens und nicht nur das Fehlen von Krankheit oder Gebrechen.“
 Verfassung der Weltgesundheitsorganisation. (PDF; 533 kB). Stand Mai 2014, S. 1 (deutsche Übersetzung)
 Axel W. Bauer: Was ist der Mensch? Antwortversuche der medizinischen Anthropologie. (Überarbeitete Version des Eröffnungsvortrags zur Tagung „Was ist der Mensch? Wie der medizinische Fortschritt das Menschenbild verändert“ der Evangelischen Akademie Baden in Bad Herrenalb vom 11. 11. 2011.) In: Fachprosaforschung – Grenzüberschreitungen 8/9, 2012/2013, ISBN 978-3-86888-077-9, S. 437–453, hier: S. 438–440 (Gesundheit und Krankheit als normative Größen der medizinischen Anthropologie).
 DIMDI: ICD-10 Homepage (Memento vom 6. März 2010 im Internet Archive)
 S. Breinersdorf: Versuch über den gegenwärtigen Standpunkt der Theorien der Medizin. Bey Iohann Friedrich Korn der Ältere (Hrsg.), 1804, S. 44, GoogleBooks
 Wolfgang Gerok, Christoph Huber, Thomas Meinertz, Henning Zeidler: Die innere Medizin. Schattauer Verlag, 2006, ISBN 3-7945-2222-2, S. 4ff, GoogleBooks
 Axel W. Bauer: Brute Facts oder Institutional Facts? Kritische Bemerkungen zum wissenschaftstheoretischen Diskurs um den allgemeinen Krankheitsbegriff. In: Erwägen – Wissen – Ethik. Band 18, 2007, Heft 1, S. 93–95.
 U. Meyer: Krankheit als leistungsauslösender Begriff im Sozialversicherungsrecht. (Memento vom 12. November 2011 im Webarchiv archive.is) In: Schweizerische Ärztezeitung. 90, 14, 2009, S. 585–588.
 BGH, Urteil vom 21. März 1958, Az.: 2 StR 393/57
 BSGE 35, 10, 12 f.
 Schweizer Bundesgesetz über den Allgemeinen Teil des Sozialversicherungsrechts, Art. 3 Krankheit, admin.ch; zuletzt eingesehen am 26. April 2009.
 Carol Gerten-Jackson: The Tuscan General Alessandro del Borro. (Memento vom 11. Oktober 2014 im Internet Archive)
 Gundolf Keil: „[…] und hat sich ein Ikterus bey Ihme angemeldet“: Krankheitsbegriffe in der Frühen Neuzeit. In: Rudolf Lenz (Hrsg.): 1976–1996: 20 Jahre Forschungsstelle für Personalschriften an der Universität Marburg – 5 Jahre Forschungsstelle für Personalschriften an der Technischen Universität Dresden. Ein Festakt. Akademie der Wissenschaften und der Literatur, Mainz 1997, S. 47–64.
 Dietrich von Engelhardt (2005), S. 797 f.
 Traute Marianne Schroeder-Kurth: Die Kulturabhängigkeit von Erkrankung - Krankheit - Kranksein - Gesundheit (Sickness - Disease - Illness - Health). Probleme global gültiger Definitionen und Konsequenzen für Erwartungen und Behandlung. In: Würzburger medizinhistorische Mitteilungen. 22, 2003, S. 306–322.
 Helmut Vogt: Das Bild des Kranken. Die Darstellung äußerer Veränderungen durch innere Leiden und ihrer Heilmaßnahmen von der Renaissance bis in unsere Zeit. München 1969. (2. Aufl. ebenda 1980)
 D. W. Haslam, W. P. James: Obesity. In: Lancet. Band 366, Nr. 9492, 2005, S. 1197–209, doi:10.1016/S0140-6736(05)67483-1, PMID 16198769.
 Traité Elémentaire de Nosologie, contenant une classification de toutes les maladies. Montpellier 1806.
 Wolfgang U. Eckart: Baumer, Jean Baptiste Thimotée. In: Werner E. Gerabek, Bernhard D. Haage, Gundolf Keil, Wolfgang Wegner (Hrsg.): Enzyklopädie Medizingeschichte. de Gruyter, Berlin/ New York 2005, ISBN 3-11-015714-4, S. 155.
 Walter Bräutigam: Psychosomatische Medizin. Ein kurzgefasstes Lehrbuch. 6. Auflage. Thieme Verlag, 1997, ISBN 3-13-498306-0, S. 245. Kapitel: Psychosomatische Gesichtspunkte bei Schwerkranken
 Max Höfler: Deutsches Krankheitsnamen-Buch. München 1899. (Neudruck Hildesheim 1970)
 Hans-Werner Altmann: Krankheitsnamen als Spiegelbild medizinischer Erkenntnisse. In: Würzburger medizinhistorische Mitteilungen. 3, 1985, S. 225–241.
 Rudolf Degkwitz u. a. (Hrsg.): Psychisch krank. S. 442.
Gesundheitshinweis	Dieser Artikel behandelt ein Gesundheitsthema. Er dient nicht der Selbstdiagnose und ersetzt keine Arztdiagnose. Bitte hierzu den Hinweis zu Gesundheitsthemen beachten!
Normdaten (Sachbegriff): GND: 4032844-2
Kategorien: KrankheitNosologieGesundheitswissenschaftWissenschaftstheorie der Biologie
Navigationsmenü
Nicht angemeldetDiskussionsseiteBeiträgeBenutzerkonto erstellenAnmeldenArtikelDiskussionLesenQuelltext anzeigenVersionsgeschichteSuche

Wikipedia durchsuchen
Hauptseite
Themenportale
Zufälliger Artikel
Mitmachen
Artikel verbessern
Neuen Artikel anlegen
Autorenportal
Hilfe
Letzte Änderungen
Kontakt
Spenden
Werkzeuge
Links auf diese Seite
Änderungen an verlinkten Seiten
Spezialseiten
Permanenter Link
Seiten­informationen
Wikidata-Datenobjekt
Artikel zitieren
Drucken/­exportieren
Buch erstellen
Als PDF herunterladen
Druckversion
In anderen Projekten
Commons
Wikinews
Wikiquote
Wikisource

In anderen Sprachen
Alemannisch
Dansk
English
Español
Français
Italiano
Nederlands
Русский
Türkçe
128 weitere
Links bearbeiten
Diese Seite wurde zuletzt am 13. November 2018 um 16:51 Uhr bearbeitet.
Abrufstatistik

Der Text ist unter der Lizenz „Creative Commons Attribution/Share Alike“ verfügbar; Informationen zu den Urhebern und zum Lizenzstatus eingebundener Mediendateien (etwa Bilder oder Videos) können im Regelfall durch Anklicken dieser abgerufen werden. Möglicherweise unterliegen die Inhalte jeweils zusätzlichen Bedingungen. Durch die Nutzung dieser Website erklären Sie sich mit den Nutzungsbedingungen und der Datenschutzrichtlinie einverstanden.
Wikipedia® ist eine eingetragene Marke der Wikimedia Foundation Inc.
DatenschutzÜber WikipediaImpressumEntwicklerStellungnahme zu CookiesMobile AnsichtWikimedia Foundation Powered by MediaWiki
"""

In [121]:
t = "@knakatani @ChikonJugular @joofford @SteveBlogs11 https://t.co/WHtaRYGNSY says lifetime risk of cervical cancer in Japan is 1 in 100.  That means HPV is endemic in Japan, and screening is not working well."
t2 = "Heute war ich beim Hausarzt und er hat festgestellt, dass es mir gut geht Haus bin ist sind uns wir die der das dem den Essen Kopf Krebs Krankheit love you the we i hate Microsoft Amazon gekauft verkauft essen trinken."
top_x(0, t3, tokenized, V)

['krankheit', 'zur', 'navigation', 'springenzur', 'suche', 'springen', 'das', 'kranke', 'mädchen', 'den', 'syge', 'pige', 'von', 'michael', 'ancher', 'krankheit', 'ist', 'ein', 'zustand', 'verminderter', 'leistungsfähigkeit', 'der', 'auf', 'funktionsstörungen', 'von', 'einem', 'oder', 'mehreren', 'organen', 'der', 'psyche', 'oder', 'des', 'gesamten', 'organismus', 'beruht', 'und', 'zurückgeht', 'diese', 'störungen', 'werden', 'ihrerseits', 'durch', 'strukturelle', 'veränderungen', 'von', 'zellen', 'und', 'geweben', 'hervorgerufen', 'die', 'lehre', 'von', 'den', 'krankheiten', 'ist', 'die', 'pathologie', 'während', 'die', 'nosologie', 'sich', 'mit', 'der', 'systematischen', 'einteilung', 'von', 'krankheiten', 'beschäftigt', 'inhaltsverzeichnis', 'wortherkunft', 'definition', 'krankheit', 'und', 'gesundheit', 'medizin', 'recht', 'geschichtliche', 'und', 'kulturelle', 'aspekte', 'typische', 'reaktionen', 'bei', 'schwerer', 'krankheit', 'systematik', 'ursachen', 'und', 'verlauf', 'krankhei

0.9632592849164086
{'a'}
1.0
{'können', 'ist', 'nicht', 'wenn', 'hin', 'am', 'für', 'zum', 'sein', 'von', 'meist', 'einen', 'man', 'schwer', 'die', 'bis', 'kann', 'es', 'ohne', 'auch'}
0.9518501587765521
set()
0
{'the', 'major', 'rate', 'in'}
0.9425502851772796
{'by', 'new'}
0.9828325155298657
{'eines'}
1.0
{'of', 'and'}
0.9999999999999999
set()
0
set()
0
{'of', 'probleme', 'the', 'a', 'and'}
0.9714935747644035
{'die', 'in'}
0.9992693586482712
{'a', 'in', 'and', 'obesity', 'by'}
0.9454424810486781
{'obesity', 'and'}
0.9999999999999998
{'obesity'}
1.0
{'bad', 'of', 'the', 'a', 'obesity'}
0.9903950151812544
{'online'}
1.0
set()
0
{'and'}
1.0
set()
0
set()
0
{'videos'}
1.0
{'wieder', 'der', 'auf', 'aber', 'das', 'am', 'ist', 'nicht', 'eine', 'einem', 'den', 'in', 'und', 'schon', 'sie', 'kann'}
0.9561528638849192
{'u'}
1.0
set()
0
set()
0
{'the'}
1.0
{'of', 'the', 'a', 'on', 'in', 'who'}
0.9621066462055818
{'selbst', 'ihrem', 'ihre', 'bei', 'in', 'sich', 'die', 'sie', 'immer', 'um', 'es', 

KeyboardInterrupt: 